<a href="https://colab.research.google.com/github/vladgap/Various/blob/main/MLNN_RELU_check_110422.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# data

In [126]:
# data=[
# [0,1,2,3,4,5,6,7,8],
# [0,0,0,-1,-2,-3,-4,-5,-6]
# ]
# data=np.array(data).T
# data=pd.DataFrame(data, columns=['sol','den']).sort_values(by=['sol'])
# data.head()

,sol,den
0,0,0
1,1,0
2,2,0
3,3,-1
4,4,-2


In [250]:
data=[
[0,1,2,3,4,5,6,7,8],
[-1,-1,1,3,5,7,9,11,13]
]
data=np.array(data).T
data=pd.DataFrame(data, columns=['sol','den']).sort_values(by=['sol'])
data.head()

,sol,den
0,0,-1
1,1,-1
2,2,1
3,3,3
4,4,5


In [127]:
pip install MLNN

# scaling

In [252]:
from sklearn import preprocessing
from MLNN import *

In [253]:
scaler_X = preprocessing.StandardScaler().fit(data[['sol']])
scaler_X.mean_, scaler_X.scale_=[0], [1]

print(scaler_X.mean_, scaler_X.scale_)
X_sc = scaler_X.transform(data[['sol']]) # matrix
X_sc[0]

[0] [1]


array([0.])

In [254]:
scaler_y = preprocessing.StandardScaler().fit(data[['den']])
scaler_y.mean_, scaler_y.scale_=[0], [1]

y_sc = scaler_y.transform(data[['den']])[:,0] # vector
y_sc

array([-1., -1.,  1.,  3.,  5.,  7.,  9., 11., 13.])

# model fitting

In [255]:
layers=[1,1,1]

In [256]:
reg1=Regres(layers=layers)
reg1.set_hidden_activ('mrelu', param=0.01)
reg1.fit(np.array(X_sc), y_sc, epochs=1000, eta=0.01)
data1=go.Scatter(y=reg1.MSE_history[:], name='eta=0.01')

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - Pandas table of weights propagation
self.MSE_history - list of MSEs propagation


In [257]:
reg2=Regres(layers=layers)
reg2.set_hidden_activ('mrelu', param=0.01)
reg2.fit(np.array(X_sc), y_sc, epochs=3000, eta=0.001)
data2=go.Scatter(y=reg2.MSE_history[:], name='eta=0.001')

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - Pandas table of weights propagation
self.MSE_history - list of MSEs propagation


In [271]:
reg_fixed=Regres(layers=layers)
reg_fixed.set_hidden_activ('mrelu', param=0.01)
reg_fixed.set_weights(fixed_weights) # if no fixed_weights exist yet, skip it
reg_fixed.fit(np.array(X_sc), y_sc, epochs=1000, eta=0.0001)
data_fixed=go.Scatter(y=reg_fixed.MSE_history[:], name='fixed')

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - Pandas table of weights propagation
self.MSE_history - list of MSEs propagation


In [272]:
fig1=go.Figure()
fig1.add_trace(data1), print('MSE1=', reg1.MSE_history[-1])
fig1.add_trace(data2), print('MSE2=', reg2.MSE_history[-1])

fig1.add_trace(data_fixed), print('MSE_fixed=', reg_fixed.MSE_history[-1])

fig1.update_yaxes(range=[0, 1])
fig1.show()

MSE1= 1.598203065356022e-14
MSE2= 1.1548044311257253
MSE_fixed= 0.0


# model fixation

In [268]:
reg1.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 1.22367782 -1.22367796] mrelu
Layer 2 Neuron 0 : [ 1.63441719 -0.99999997] linear



In [269]:
reg1.weight_history[-1]

[[[1.2236778241570374, -1.223677964309536]],
 [[1.6344171886277459, -0.999999973706667]]]

In [297]:
reg1.regres_network.network[1][0].weights[0]=1.2237
reg1.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 1.2237     -1.22367796] mrelu
Layer 2 Neuron 0 : [ 1.63441719 -0.99999997] linear



In [270]:
fixed_weights=[[[1, -1]],
 [[2, -1]]]
fixed_MSE=0

# predictions

In [279]:
chosen_model=reg_fixed
# chosen_model=reg2

In [280]:
predics=scaler_y.inverse_transform([chosen_model.predict(np.array(X_sc))])[0]
errors=(predics-data.den)/data.den*100

In [281]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=data.sol, y=data.den, mode='markers', name='data'),secondary_y=False)
fig.add_trace(go.Scatter(x=data.sol, y=predics, mode='lines', name='prediction'),secondary_y=False)
fig.add_trace(go.Scatter(x=data.sol, y=errors, mode='markers', name='errors'),secondary_y=True)

fig.update_xaxes(title_text="R(Sol)")

# Set y-axes titles
fig.update_yaxes(title_text="Density", secondary_y=False)
fig.update_yaxes(title_text="% error", range=[-5, 5], secondary_y=True, showgrid=False, tick0=-5, dtick=2)

fig.show()

In [282]:
chosen_model.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 1. -1.] mrelu
Layer 2 Neuron 0 : [ 2. -1.] linear



In [283]:
print('X mean=', scaler_X.mean_,',X stdev=', scaler_X.scale_)
print('y mean=', scaler_y.mean_[0],',y stdev=', scaler_y.scale_[0])

X mean= [0] ,X stdev= [1]
y mean= 0 ,y stdev= 1
